In [1]:
# import the packages needed
import redis
import csv
import pandas as pd
import numpy as np
import re

import psycopg2

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

from py2neo import Graph, Node, Relationship

import pandas as pd
import numpy as np

from rake_nltk import Rake
from nltk.corpus import stopwords 

import spacy

In [2]:
# set the default magic numbers
DEFAULT_TTL = 300
DEFAULT_RAKE_CUTOFF = 3

In [3]:
# import the dataframe used
leg_cur_df = pd.read_csv('./data/raw/legislators-current.csv')
cities_df = pd.read_csv("./data/raw/us_cities.csv")[["city", "cityWithAccent", "stateProvince"]]
usnews_df = pd.read_csv("./data/raw/usnews.csv")

In [4]:
# set up the connection of the databases
conn = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="Lsd10234"
)
cur = conn.cursor()


r =  redis.Redis(host='localhost', port=6379, db=0, decode_responses=True) # start new client with decoder

graph = Graph("bolt://localhost:7687", user = "neo4j", password = "password")

In [5]:
# load legistlator data into postgres

sub_query = ''
for key, val in dict(leg_cur_df.dtypes).items():
    if val == 'object':
        postgres_dtype = "varchar"
    else:
        postgres_dtype = "integer"
    
    sub_query += ' {} {},'.format(key, postgres_dtype)
    

sub_query += 'PRIMARY KEY (last_name, first_name)'

cur.execute("DROP TABLE IF EXISTS legislators")

cur.execute("CREATE TABLE legislators ({});".format(sub_query))

copy_sql = """
           COPY legislators FROM stdin WITH CSV HEADER
           DELIMITER as ','
           """

with open('./data/raw/legislators-current.csv') as f:
    cur.copy_expert(sql=copy_sql, file=f)
    conn.commit()

In [6]:
# load us_cities data into postgres

sub_query = ''
for key, val in dict(cities_df.dtypes).items():
    postgres_dtype = "varchar"
    sub_query += ' {} {},'.format(key, postgres_dtype)
    

sub_query += 'PRIMARY KEY (stateProvince, city)'

cur.execute("DROP TABLE IF EXISTS cities")

cur.execute("CREATE TABLE cities ({});".format(sub_query))

copy_sql = """
           COPY cities FROM stdin WITH CSV HEADER
           DELIMITER as ','
           """

with open('./data/raw/us_cities.csv') as f:
    cur.copy_expert(sql=copy_sql, file=f)
    conn.commit()

In [7]:
# load us_cities data into postgres

sub_query = ''
for key, val in dict(cities_df.dtypes).items():
    postgres_dtype = "varchar"
    sub_query += ' {} {},'.format(key, postgres_dtype)
    

sub_query += 'PRIMARY KEY (stateProvince, city)'

cur.execute("DROP TABLE IF EXISTS cities")

cur.execute("CREATE TABLE cities ({});".format(sub_query))

copy_sql = """
           COPY cities FROM stdin WITH CSV HEADER
           DELIMITER as ','
           """

with open('./data/raw/us_cities.csv') as f:
    cur.copy_expert(sql=copy_sql, file=f)
    conn.commit()

In [8]:
# load usnews data into postgres

sub_query = ''
for key, val in dict(usnews_df.dtypes).items():
    postgres_dtype = "varchar"
    sub_query += ' {} {},'.format(key, postgres_dtype)
    

sub_query += 'PRIMARY KEY (id)'

cur.execute("DROP TABLE IF EXISTS usnews")

cur.execute("CREATE TABLE usnews ({});".format(sub_query))

copy_sql = """
           COPY usnews FROM stdin WITH CSV HEADER
           DELIMITER as ','
           """

with open('./data/raw/usnews.csv', encoding='utf-8') as f:
    cur.copy_expert(sql=copy_sql, file=f)
    conn.commit()

In [9]:
# set redis config to 2mb size and use LRU to evict keys
r.config_set('maxmemory', '256mb')
r.config_set('maxmemory-policy', 'volatile-ttl')

True

In [10]:
# create the cache query method
def redisLRU_get_pg(query, cur, r):
    '''
    Takes in query in the form of "SELECT a, b, c FROM legislators WHERE x AND y AND z"
    inputs:
        query: PostgreSQL query string
        cur: postgres cursor
        r: redis connection
    output:
        iterable of query results
        [(result_11, ..., result_1n), ..., (result_n1, ..., resutl_nn)]
    '''
    
    # patch sql query into redis handlable key
    out_cols, _ , condition_str = query.replace('SELECT', '').partition('FROM')
    out_cols = [n.strip() for n in out_cols.split(',')]
    conditions = [n.strip() for n in condition_str.partition('WHERE')[-1].split('AND')]
    rkey = []
    try:
        for condition in conditions:
            cond_col, cond_val = condition.replace('=', ' ').split()
            rkey.append(cond_col + ':' + cond_val)
        rkey = ','.join(rkey)
    except:
        rkey = 'all'


    # if no restrictions are provided, its better just to query db due to its impact on cache
    if rkey == 'all':
        cur.execute(query)
        return cur.fetchall()

    outs = []
    for out_col in out_cols:
        redis_out = r.hget(out_col, rkey)
        if redis_out == None:
            cur.execute("SELECT {} FROM {}".format(out_col, condition_str))
            postgres_out = cur.fetchall()
            r.hset(rkey, out_col, str(postgres_out))
            r.expire(rkey, DEFAULT_TTL)
            outs.append(postgres_out)
        else:
            r.expire(rkye, DEFAULT_TTL)
            outs.append(eval(redis_out))

    return np.hstack(outs)

In [11]:
# test redislru: assume the queries are highly similar
def redisLRU_get_pg_low_prec(query, cur, r):
    if r.ttl(query) == -2:
        cur.execute(query)
        out = cur.fetchall()
        r.set(query, str(out), ex=DEFAULT_TTL)
    else:
        r.expire(query, DEFAULT_TTL)
        out = eval(r.get(query))
    return out

In [12]:
# test 1
query = "SELECT last_name, first_name, party, state FROM legislators WHERE state = 'CA' AND first_name = 'John'"
cur.execute(query)
cur.fetchall()

[('Garamendi', 'John', 'Democrat', 'CA')]

In [13]:
# redislru
redisLRU_get_pg(query, cur, r)

array([['Garamendi', 'John', 'Democrat', 'CA']], dtype='<U9')

In [14]:
# test redislru
redisLRU_get_pg_low_prec(query, cur, r)

[('Garamendi', 'John', 'Democrat', 'CA')]

In [15]:
%%time
# test speed
query = "SELECT * FROM legislators WHERE state = 'CA' AND first_name = 'John'"
cur.execute(query)
cur.fetchall()

Wall time: 1.03 ms


[('Garamendi',
  'John',
  None,
  None,
  None,
  'John Garamendi',
  '1945-01-24',
  'M',
  'rep',
  'CA',
  3,
  None,
  'Democrat',
  'https://garamendi.house.gov',
  '2368 Rayburn House Office Building; Washington DC 20515-0503',
  '202-225-1880',
  None,
  'http://garamendi.house.gov/rss.xml',
  'RepGaramendi',
  'repgaramendi',
  'garamendiCA10',
  'UCRym1qsO72C39gX6pBu4icg',
  'G000559',
  1973,
  'N00030856',
  None,
  'H0CA10149',
  18413,
  412382,
  29664,
  None,
  None,
  20958,
  'John Garamendi')]

In [16]:
%%time
# test speed of redislru
redisLRU_get_pg(query, cur, r)

Wall time: 2.84 ms


array([['Garamendi', 'John', None, None, None, 'John Garamendi',
        '1945-01-24', 'M', 'rep', 'CA', 3, None, 'Democrat',
        'https://garamendi.house.gov',
        '2368 Rayburn House Office Building; Washington DC 20515-0503',
        '202-225-1880', None, 'http://garamendi.house.gov/rss.xml',
        'RepGaramendi', 'repgaramendi', 'garamendiCA10',
        'UCRym1qsO72C39gX6pBu4icg', 'G000559', 1973, 'N00030856', None,
        'H0CA10149', 18413, 412382, 29664, None, None, 20958,
        'John Garamendi']], dtype=object)

In [17]:
query = "SELECT last_name, first_name, party FROM legislators WHERE state = 'NA'"

In [18]:
%%time
# test speed of original query
cur.execute(query)
out = cur.fetchall()

Wall time: 2.98 ms


In [19]:
%%time
# test speed of redislru
out = redisLRU_get_pg(query, cur, r)

Wall time: 6.28 ms


In [20]:
%%time
# test speed of test redislru
out = redisLRU_get_pg_low_prec(query, cur, r)

Wall time: 2.4 ms


In [21]:
# apply spacy on the news content, and populate neo4j
counter = spacy.load('en_core_web_sm')
query = "SELECT * FROM usnews"
out = redisLRU_get_pg_low_prec(query, cur, r)

def spacy_count(text):
    if text != None:
        ents = counter(text).ents
        return set([(p_ent.text.lower(), p_ent.label_) for p_ent in ents if (p_ent.label_ == 'PERSON' or p_ent.label_ == 'GPE')])
    else:
        return set()
    
def create_nodes_from_news(news):
    n1 = Node("news", news_id = news[0], date = news[1], title = news[2], content = news[3])
    text = news[3]
    graph.create(n1)
    for entity in spacy_count(text):
        n2 = Node(entity[1].lower(), name = entity[0])
        rel = Relationship(n1, "mentioned", n2)
        tx = graph.begin()
        tx.merge(n2, entity[1].lower(), "name")
        tx.merge(rel)
        tx.commit()

In [29]:
# delete the previous records, and create constraints on the node
graph.delete_all()
if not graph.schema.get_uniqueness_constraints('person'):
    graph.schema.create_uniqueness_constraint('person', 'name')
if not graph.schema.get_uniqueness_constraints('gpe'):
    graph.schema.create_uniqueness_constraint('gpe', 'name')
if not graph.schema.get_uniqueness_constraints('news'):
    graph.schema.create_uniqueness_constraint('news', 'news_id')

In [30]:
# populate the nodes
for news in out:
    create_nodes_from_news(news)

In [107]:
# the method to check the short path between the two nodes
def shortest_path(first = "Kyrsten Sinema", second = "new york"):
    q = f"""
    match (n), (f)
    where n.name = "{first.lower()}" AND f.name = "{second.lower()}"
    return n.name as first_node_name, length(shortestPath((n)-[*]-(f))) as shortest_path, f.name as second_node_name
    order by shortest_path desc
    """
    res = graph.run(q)
    return res

In [108]:
# Default example of shortest path
shortest_path()

 first_node_name | shortest_path | second_node_name 
-----------------|---------------|------------------
 kyrsten sinema  |             4 | new york         

In [110]:
# example of shortest path between default first node and tucson
shortest_path(second = "tucson")

 first_node_name | shortest_path | second_node_name 
-----------------|---------------|------------------
 kyrsten sinema  |             4 | tucson           
 kyrsten sinema  |             4 | tucson           

In [112]:
# example of shortest path between Donald Trump and New York
shortest_path(first = "Donald Trump", second = "New York")

 first_node_name | shortest_path | second_node_name 
-----------------|---------------|------------------
 donald trump    |             4 | new york         
 donald trump    |             2 | new york         

In [59]:
# The method to get the possible legislators in nth neighborhood
def get_possible_legislator_in_n_neighborhood(name_string = "Kyrsten Sinema", n=2):
    all_person = graph.run(
        """MATCH (p:person)-[*1..{}]-(n:person) WHERE p.name CONTAINS "{}" RETURN DISTINCT n.name""".format(
            n, name_string.lower()
        )
    ).to_data_frame()
    names = all_person.iloc(axis=1)[0]
    count = 0
    out_names = set()
    for name in names:
        out_name = set()
        name = name.replace('’s','')
        sub_names = name.split(' ')
        if len(sub_names) == 2:
            query = """SELECT first_name, last_name FROM legislators 
            WHERE first_name='{}' AND last_name='{}'""".format(sub_names[0].capitalize(), sub_names[1].capitalize())
            
            out_name.update([tuple(name) for name in redisLRU_get_pg(query, cur, r)])
        else:
            for sub_name in sub_names:
                if '.' in sub_name:
                    continue
                sub_name = sub_name.capitalize()
                query = """SELECT first_name, last_name FROM legislators 
                WHERE last_name='{}' OR first_name='{}'""".format(sub_name, sub_name)
                q_results = redisLRU_get_pg(query, cur, r)
                if len(out_name) == 0:
                    out_name.update(q_results)
                else:
                    out_name.intersection_update(q_results)
        out_names.update(out_name)
    return out_names

In [64]:
# testing of second neighborhood of Donald
test=get_possible_legislator_in_n_neighborhood("Donald", n=2)
test

{('Abby', 'Finkenauer'),
 ('Abigail', 'Spanberger'),
 ('Adam', 'Smith'),
 ('Adrian', 'Smith'),
 ('Al', 'Green'),
 ('Alma', 'Adams'),
 ('Amy', 'Klobuchar'),
 ('Andy', 'Harris'),
 ('Andy', 'Kim'),
 ('Angus', 'King'),
 ('Ann', 'Kirkpatrick'),
 ('Ann', 'Kuster'),
 ('Ann', 'Wagner'),
 ('Anthony', 'Brown'),
 ('Antonio', 'Delgado'),
 ('Austin', 'Scott'),
 ('Barbara', 'Lee'),
 ('Barry', 'Loudermilk'),
 ('Ben', 'Cline'),
 ('Ben', 'LujÃ¡n'),
 ('Ben', 'McAdams'),
 ('Bennie', 'Thompson'),
 ('Bernard', 'Sanders'),
 ('Bill', 'Cassidy'),
 ('Bill', 'Flores'),
 ('Bill', 'Foster'),
 ('Bill', 'Huizenga'),
 ('Bill', 'Johnson'),
 ('Bill', 'Pascrell'),
 ('Bill', 'Posey'),
 ('Buddy', 'Carter'),
 ('Carol', 'Miller'),
 ('Carolyn', 'Maloney'),
 ('Cedric', 'Richmond'),
 ('Charles', 'Grassley'),
 ('Charles', 'Schumer'),
 ('Charlie', 'Crist'),
 ('Chip', 'Roy'),
 ('Chris', 'Coons'),
 ('Christopher', 'Murphy'),
 ('Christopher', 'Smith'),
 ('Collin', 'Peterson'),
 ('Conor', 'Lamb'),
 ('Cory', 'Booker'),
 ('Cory', 'Ga

In [65]:
# size of the result
len(test)

273

In [62]:
# example possible legislators of Donald's fourth neighborhood
test=get_possible_legislator_in_n_neighborhood("Donald", n=4)
test

{('A.', 'Ferguson'),
 ('Abby', 'Finkenauer'),
 ('Abigail', 'Spanberger'),
 ('Adam', 'Kinzinger'),
 ('Adam', 'Schiff'),
 ('Adam', 'Smith'),
 ('Adrian', 'Smith'),
 ('Al', 'Green'),
 ('Al', 'Lawson'),
 ('Alan', 'Lowenthal'),
 ('Alcee', 'Hastings'),
 ('Alex', 'Mooney'),
 ('Alma', 'Adams'),
 ('Ami', 'Bera'),
 ('Amy', 'Klobuchar'),
 ('AndrÃ©', 'Carson'),
 ('Andy', 'Biggs'),
 ('Andy', 'Harris'),
 ('Andy', 'Kim'),
 ('Andy', 'Levin'),
 ('Angie', 'Craig'),
 ('Angus', 'King'),
 ('Ann', 'Kirkpatrick'),
 ('Ann', 'Kuster'),
 ('Ann', 'Wagner'),
 ('Anna', 'Eshoo'),
 ('Anthony', 'Brindisi'),
 ('Anthony', 'Brown'),
 ('Anthony', 'Gonzalez'),
 ('Antonio', 'Delgado'),
 ('Austin', 'Scott'),
 ('Ayanna', 'Pressley'),
 ('Barbara', 'Lee'),
 ('Barry', 'Loudermilk'),
 ('Ben', 'Cline'),
 ('Ben', 'LujÃ¡n'),
 ('Ben', 'McAdams'),
 ('Benjamin', 'Cardin'),
 ('Benjamin', 'Sasse'),
 ('Bennie', 'Thompson'),
 ('Bernard', 'Sanders'),
 ('Betty', 'McCollum'),
 ('Bill', 'Cassidy'),
 ('Bill', 'Flores'),
 ('Bill', 'Foster'),
 ('

In [63]:
# length of the example result
len(test)

451

In [91]:
# method to get the local clustering coefficient of specified name of the person or city
def n_neighborhood_lcc(name_string, n=2):
    query = """MATCH (p:person)-[r*1..{}]-(n) 
    WHERE p.name CONTAINS "{}" 
    RETURN count(DISTINCT r), count(DISTINCT n);""".format(n, name_string.lower())
    r, n = graph.run(query).to_data_frame().iloc[0].array
    #excluded center node, thus n*(n+1)
    return r/(n*(n+1))

In [93]:
# lcc of Donald Trump
n_neighborhood_lcc("donald trump")

0.001641524995344135

In [94]:
# lcc of Kyrsten Sinema
n_neighborhood_lcc("kyrsten sinema")

0.09941520467836257

In [111]:
# lcc of Bill Stepien
n_neighborhood_lcc("Bill Stepien")

0.009219025249559601

In [187]:
cur.close()
conn.close()